### Notebook to parse text files to produce cleaned text of RPD (Legacy) decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0). 

Dataset & Code to be cited as:

Sean Rehaag, "Refugee Protection Division (Legacy) Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/rpd/>.

Notes:

(1) Data Source: In the fall of 2022 the Immigration and Refugee Board provided the RLL with a full backlog of approximately 116k published decisions from all divisions (RAD, RPD, ID, IAD). Because the IRB no longer regularly publishes RPD decisions, the dataset is no longer being updated, which is why we refer to the 
dataset as a legacy dataset. For more recent RPD decisions (obtained via Access to Information Requests), 
see the RLLR dataset.

(2) Unofficial Data: The data are unofficial reproductions. For official versions, please contact the Immigration and Refugee Board. 

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Immigration and Refugee Board.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, please contact the Immigration and Refugee Board. 

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information.

Acknowledgements: Thanks to Rafael Dolores who coded the initial parsing scripts for the Refugee Appeal Division Bulk Decisions Dataset, which were modified for this datset.


# Installing Libraries

In [ ]:
#!pip install langdetect
#!pip install regex
#!pip install dask

# Importing Libraries

In [1]:
import os
import regex as re 
import pandas as pd
from datetime import datetime
from langdetect import detect, DetectorFactory
from difflib import get_close_matches
import json
from tqdm import tqdm
import chardet
import dask.bag as db
from dask.diagnostics import ProgressBar
import pathlib
from pathlib import Path
from datetime import datetime, timezone

## Declaring Constant
Here, we specify the directories containing our data files.

In [2]:
DATA_DIRS = ["../IRB Decisions - Initial Request - TEXT"]

# For SR:
DATA_DIRS = ["d:/IRB Decisions - Initial Request - TEXT/"]

In [3]:
# set seed for langdetect for consistent results and reproducibility
DetectorFactory.seed = 42



## Language Detection
This function determines the language of a given text.

In [4]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return None


## Regular Expression Detector
Functions to parse the date from text files while accounting for several different formats

In [5]:
def original_match_date_patterns(content):
    patterns = {
        "custom": (r"Date (?:of decision|de la décision)\s*\n\s*([A-Za-z]+)\s+(\d{1,2})\.\s*(\d{4})", lambda m: [m.group(1), m.group(2), m.group(3)]),
        "primary": (r"Date (?:of decision|de la décision)\s*(?:Le )?\s*((?:(?:\d{1,2}|1er)\s+[\w]+\s*,?\s*\d{1,4})|\w+\s+\d{1,2}(?:st|nd|rd|th)?\s*,?\s*\d{1,4}|\d{1,2}-\d{1,2}-\d{1,4})", lambda m: m.group(1).replace(',', '').split()),
        "original_decision": (r"Date of decision\s+([\w\s]+),\s+(\d{4})\s+\(original decision\)", lambda m: m.group(1).strip().split() + [m.group(2).strip()]),
        "tribunal": (r"Tribunal\s*\n\s*([\w\s]+?)\s*\n\s*Date of decision", lambda m: m.group(1).replace(',', '').split()),
        "original": (r"Original\s+([\w]+\s+\d{1,2}(?:st|nd|rd|th)?,?\s+\d{4})", lambda m: m.group(1).replace(',', '').split())
    }

    for key, (pattern, process) in patterns.items():
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            return process(match)
    return None

def match_date_patterns(content):
    # 1. Try your proven patterns first
    result = original_match_date_patterns(content)
    if result:
        return result

    # 2. Fallback: Check lines after the label
    lines = content.splitlines()
    label_regex = re.compile(
        r'^\s*Date (of decision|de la décision)(\s*and reasons|\s*et des motifs)?\s*$', re.IGNORECASE)

    date_regexes = [
        # French style: 1er mai 2015, 24 juillet 2018, Le 24 juillet 2018
        re.compile(r'^(Le\s+)?(\d{1,2}|1er)\s+\w+\s+\d{4}$', re.IGNORECASE),
        # English style: January 21, 2019 or November 5th, 2014
        re.compile(r'^[A-Za-z]+\s+\d{1,2}(st|nd|rd|th)?\,?\s+\d{4}$'),
        # Dashes (rare but sometimes): 10-12-2018
        re.compile(r'^\d{1,2}-\d{1,2}-\d{4}$'),
    ]

    for idx, line in enumerate(lines):
        if label_regex.match(line):
            # Look ahead at the next 3 non-empty lines
            lookahead = 0
            for j in range(idx+1, min(idx+5, len(lines))):
                candidate = lines[j].strip()
                if not candidate:
                    continue
                lookahead += 1
                for reg in date_regexes:
                    m = reg.match(candidate)
                    if m:
                        cleaned = candidate.replace("Le ", "").replace(",", "")
                        return cleaned.split()
                if lookahead >= 3:
                    break
    
    return None

## Date Formatter
Takes detected regular expression and converts into one common format

In [6]:
french_to_english = {
        'janvier': 'January', 'fevrier': 'February', 'mars': 'March', 'avril': 'April',
        'mai': 'May', 'juin': 'June', 'juillet': 'July', 'aout': 'August',
        'septembre': 'September', 'octobre': 'October', 'novembre': 'November', 'decembre': 'December'
}

def correct_month_name(misspelled_month, possibilities=['Janvier','January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], cutoff=0.6):
    correct_months = get_close_matches(misspelled_month, possibilities, n=1, cutoff=cutoff)
    if correct_months:
        corrected_month = correct_months[0]
        # Check if the corrected month is in the French to English mapping
        return french_to_english.get(corrected_month.lower(), corrected_month)
    else:
        return misspelled_month

def correct_year_typo(year):
    if len(year) == 3 and year.startswith("0"):
        return "20" + year[1:]
    return year

def correct_year_typo(year):
    """Corrects year format typos."""
    return "20" + year[1:] if len(year) == 3 and year.startswith("0") else year

def process_numeric_format(parts):
    """Processes numeric date format 'dd-mm-yyyy'."""
    day, month, year = parts[0].split('-')
    year = correct_year_typo(year)
    return datetime(int(year), int(month), int(day)).date().strftime('%Y-%m-%d')

def process_day_first_format(parts, french_month_mapping):
    """Processes dates in 'day month year' format, French or English."""
    day = 1 if parts[0].lower() == '1er' else int(parts[0])

    month = ''
    # Check if month and year are concatenated
    if len(parts) == 2 and not parts[1].isdigit():
        month_year_str = parts[1]
        for i in range(1, len(month_year_str)):
            if month_year_str[i:].isdigit():
                month_str, year_str = month_year_str[:i], month_year_str[i:]
                year = correct_year_typo(year_str)
                month = french_month_mapping.get(month_str.lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o'), month_str.capitalize())
                break
    else:
        month = parts[1].lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o')
        if len(parts) >= 3:
            year = correct_year_typo(parts[2])

    if month in french_month_mapping:
        return datetime(int(year), french_month_mapping[month], day).date().strftime('%Y-%m-%d')
    else:
        if isinstance(month, int):
            return datetime(int(year), month, day).date().strftime('%Y-%m-%d')
        
        corrected_month = correct_month_name(month.capitalize())
        try:
            return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
        except ValueError as e:
            print(f"Error parsing date: {e}")
            return None

def process_month_first_format(parts):
    """Processes month first format with possible ordinal suffix."""
    day = 0
    month = ''
    year = ''
    
    if len(parts) == 2 and parts[1].isdigit() and len(parts[1]) > 2:
        
        if parts[1].isdigit() and len(parts[1]) > 4: 
            month = parts[0]
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = year_str
            day = int(day_str)
            
        elif parts[1].isdigit()and len(parts[1]) > 3: #Year is the second entry
            month_day_str = parts[0]
            for i in range(1, len(month_day_str)):
                if not month_day_str[i].isdigit():
                    day_str, month_str = month_day_str[:i], month_day_str[i:]
                    if day_str.isdigit():
                        day = int(day_str)
                    month = french_to_english.get(month_str.lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o'), month_str)
                    parts[0] = month
                    break
            year = parts[1]
        else:
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = correct_year_typo(year_str)
            day = int(day_str)

    else:
        day = re.sub(r"[^\d]", "", parts[1])
        day = int(day) if day.isdigit() else 1
        if len(parts) >= 3:
            year = correct_year_typo(parts[2])
            
    try:
        corrected_month = correct_month_name(parts[0].capitalize())
        return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None


## Document Date Extraction
This function searches the given file for the document date using regular expressions, taking into account both French and English texts.

In [7]:
def process_date_parts(parts, french_month_mapping):
    """Determines the correct date processing method based on the format of the parts."""
    if '-' in parts[0]:
        return process_numeric_format(parts)
    elif parts[0].isdigit() or parts[0].lower() == '1er':
        return process_day_first_format(parts, french_month_mapping)
    else:
        return process_month_first_format(parts)

def extract_document_date(content):
    french_month_mapping = {
        'janvier': 1, 'fevrier': 2, 'mars': 3, 'avril': 4,
        'mai': 5, 'juin': 6, 'juillet': 7, 'aout': 8,
        'septembre': 9, 'octobre': 10, 'novembre': 11, 'decembre': 12
    }
    
    parts = match_date_patterns(content)
    
    if not parts:
        return None
    return process_date_parts(parts, french_month_mapping)

## File Processor Helpers

In [8]:
def extract_rpd_number(content):
    """Extracts the RPD number from the content, ignoring IAD/ID/RAD files."""
    # Check for lines indicating the file should be ignored
    ignore_lines = ["IAD File",
                    "dossier de la SAI",
                    "IMMIGRATION APPEAL DIVISION", 
                    "ID File", 
                    "IMMIGRATION DIVISION", 
                    "RAD File",
                    "REFUGEE APPEAL DIVISION",
                    "REFUGEE DIVISION"
                    ]
    
    for line in content.splitlines():
        # Sanitize the current line
        sanitized_line = ''.join(c for c in line if c.isprintable()).strip()

        # Check if the sanitized line matches any ignore line
        if any(ignore_line in sanitized_line for ignore_line in ignore_lines):   
            return None
        
        if "RPD File" in sanitized_line or "RPD file" in sanitized_line :          
            rpd_number_match = re.search(r"([A-Z]{2}\d+.\d+)", sanitized_line)
            if rpd_number_match:
                return rpd_number_match.group(1)
            
            # If RPD is in the next immediate line
            next_line_index = content.splitlines().index(line) + 1
            if next_line_index < len(content.splitlines()):
                next_line = content.splitlines()[next_line_index]
                rpd_number_match = re.search(r"([A-Z]{2}\d+-\d+)", next_line)
                if rpd_number_match:
                    return rpd_number_match.group(1)

    if "REFUGEE PROTECTION DIVISION" in content or "SECTION DE LA PROTECTION DES RÉFUGIÉS" in content:
        rpd_number_match = re.search(r"([A-Z]{2}\d+-\d+)", content)
        if rpd_number_match:
            return rpd_number_match.group(1)

    return None

def process_file(file_path):
    """Processes a single file and extracts data."""

    # manually exclude problematic files
    problem_files = ["1821419", "636351"]
    if Path(file_path).stem in problem_files:
        return None
    
    # Use chardet to detect the encoding of the file
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    encoding = chardet.detect(raw_data)['encoding']

    # Read the file with the detected encoding
    with open(file_path, 'r', errors='replace', encoding=encoding) as file:
        content = file.read()

    rpd_number = extract_rpd_number(content)
    if rpd_number:
        lang = detect_language(content)
        document_date = extract_document_date(content)
        
        scraped_timestamp = datetime.fromtimestamp(os.path.getmtime(file_path), tz=timezone.utc)


        return {
            'citation': rpd_number,
            'citation2': '',
            'dataset': 'RPD',
            'name': '',
            'language': lang,
            'document_date': document_date,
            'url': os.path.basename(file_path),
            'scraped_timestamp': scraped_timestamp,
            'referrer': 'a2aj_parse_rpd_txt.ipynb',
            'unofficial_text': content,
            
        }
    return None

## Processing Files
This block of code reads each file in the dataset directories to extract the needed information, using the previously defined functions and form a Pandas dataframe.

In [9]:
# Main data processing loop (run in paraellel using Dask)

def process_file_wrapper(file_path):
    if not os.path.basename(file_path).startswith('~'):
        return process_file(file_path)

# Gather all file paths
file_paths = []
for data_dir in DATA_DIRS:
    if os.path.exists(data_dir) and os.path.isdir(data_dir):
        dir_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
        file_paths.extend(dir_files)

# Create a Dask Bag from file paths
file_bag = db.from_sequence(file_paths)

# Use Dask to process files in parallel
with ProgressBar():
    results = file_bag.map(process_file_wrapper).filter(lambda x: x is not None).compute()

# Convert results to a Pandas DataFrame
df = pd.DataFrame(results)

# # Process files using a regular loop for easier debugging if needed
# results = []
# for file_path in file_paths:
#     try:
#         result = process_file_wrapper(file_path)
#         if result is not None:
#             results.append(result)
#     except Exception as e:
#         print(f"Error processing file {file_path}: {e}")

# # Convert results to a Pandas DataFrame
# df = pd.DataFrame(results)

[########################################] | 100% Completed | 227.64 s


In [10]:
df_backup = df.copy()

In [ ]:
df = df_backup.copy()

### Data cleaning
Cleans data to match huggingface dataset

In [11]:
# Manually fix some dates:
print ("Starting df", len(df))
# if df.doccument_date are in a series of problem dates, then drop the row from the df
problem_dates = ["0013-10-11",
                 "0013-05-31", 
                 "3006-03-03", 
                 "1005-10-04", 
                 "3013-08-13",
                 "1015-06-03",
                 "1006-03-31",                 
                 ]
df = df[~df['document_date'].isin(problem_dates)].reset_index(drop=True)

# manually remove some files
problem_files = ["1025049.txt",
                 "1529910.txt",
                 "1742145.txt",
                 "638230.txt",
                 "1024036.txt",
                 "1025063.txt",
                 "1742152.txt",
                 "547985.txt",
                 "638231.txt",
                 "1529916.txt",
                ]
df = df[~df['url'].isin(problem_files)].reset_index(drop=True)

# manually fixe some languages
df.loc[df['url'] == '1095022.txt', 'language'] = 'fr'

# manually fix some document dates
df.loc[df['url'] == '1522298.txt', 'document_date'] = '2023-11-13' 
df.loc[df['url'] == '1547144.txt', 'document_date'] = '2006-09-29' 
df.loc[df['url'] == '1783693.txt', 'document_date'] = '2012-06-04' 
df.loc[df['url'] == '1813165.txt', 'document_date'] = '2013-09-17' 
df.loc[df['url'] == '955532.txt', 'document_date'] = '2008-11-20' 




print("After removing problem dates and files:", len(df))

# fix dates format
df['document_date'] = pd.to_datetime(df['document_date'], errors='coerce', utc=True)

# Remove rows where unofficial text is duplicated, keeping the last one, sorted by scraped_timestamp
# sort by scraped_timestamp
df = df.sort_values(by='scraped_timestamp', ascending=True)
df = df.drop_duplicates(subset=['unofficial_text'], keep='last').reset_index(drop=True)
print ("After removing duplicate unofficial text", len(df))

# print number of rows with no document_date
print("Number of rows with no document_date:", df['document_date'].isna().sum())

# export to parquet the rows where there is no document_date
df_no_date = df[df['document_date'].isna()]
df_no_date.to_parquet("DATA/df_no_date.parquet", index=False)

# Remove rows where year document_date is before 2001 or after 2020
df = df[df["document_date"].dt.year.between(2002, 2020, inclusive="both")].reset_index(drop=True)
print ("After removing before 2002 and after 2020", len(df)) 


df



Starting df 13838
After removing problem dates and files: 13823
After removing duplicate unofficial text 13818
Number of rows with no document_date: 356
After removing before 2002 and after 2020 13362


,citation,citation2,dataset,name,language,document_date,url,scraped_timestamp,referrer,unofficial_text
0,MA6-05104,,RPD,,fr,2008-04-04 00:00:00+00:00,1005371.txt,2023-11-12 22:50:45.304017+00:00,a2aj_parse_rpd_txt.ipynb,Commission de l'immigration et du statut de ré...
1,TA6-12745,,RPD,,en,2008-07-03 00:00:00+00:00,1005377.txt,2023-11-12 22:50:45.398529+00:00,a2aj_parse_rpd_txt.ipynb,\n\n\nRPD File No. / N° de dossier de la SPR :...
2,TA8-21155,,RPD,,fr,2009-10-27 00:00:00+00:00,1005397.txt,2023-11-12 22:50:45.497039+00:00,a2aj_parse_rpd_txt.ipynb,\n\n\n\nN° de dossier de la SPR/RPD File No.: ...
3,MA6-00156,,RPD,,fr,2008-06-06 00:00:00+00:00,1005435.txt,2023-11-12 22:50:45.576555+00:00,a2aj_parse_rpd_txt.ipynb,Commission de l'immigration\net du statut de r...
4,TA7-02522,,RPD,,en,2009-09-17 00:00:00+00:00,1005448.txt,2023-11-12 22:50:45.662073+00:00,a2aj_parse_rpd_txt.ipynb,\n\nRPD File No. / N° de dossier de la SPR : ...
...,...,...,...,...,...,...,...,...,...,...
13357,TA6-11195,,RPD,,fr,2008-06-04 00:00:00+00:00,993958.txt,2023-11-13 18:39:26.698455+00:00,a2aj_parse_rpd_txt.ipynb,\n\n\nN° de dossier de la SPR/RPD File No.: TA...
13358,TA7-14136,,RPD,,en,2008-12-02 00:00:00+00:00,993959.txt,2023-11-13 18:39:26.852963+00:00,a2aj_parse_rpd_txt.ipynb,\n\n\nRPD File No. / N° de dossier de la SPR :...
13359,TA7-10081,,RPD,,fr,2008-08-15 00:00:00+00:00,993961.txt,2023-11-13 18:39:26.993989+00:00,a2aj_parse_rpd_txt.ipynb,\n\n\nN° de dossier de la SPR/RPD File No.: TA...
13360,MA7-01235,,RPD,,en,2010-12-13 00:00:00+00:00,997313.txt,2023-11-13 18:39:57.240724+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and\nRefugee Board\nRefugee Protec...


In [12]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # # Remove single newlines
    # text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

tqdm.pandas()
df['unofficial_text'] = df.unofficial_text.progress_apply(clean_text)

100%|██████████| 13362/13362 [00:18<00:00, 737.11it/s]


In [ ]:
# get df_en where language is English
df_en = df[df['language'] == 'en'].reset_index(drop=True)
# get df_fr where language is French
df_fr = df[df['language'] == 'fr'].reset_index(drop=True)

# print how many df_en.citation is duplicated
print("Number of duplicated citations in df_en:", df_en.citation.duplicated().sum())
print("Number of duplicated citations in df_fr:", df_fr.citation.duplicated().sum())

# print how many rows in df_en are duplicated for BOTH citation and document_date
print("Number of duplicated rows in df_en for BOTH citation and document_date:", df_en.duplicated(subset=['citation', 'document_date']).sum())
print("Number of duplicated rows in df_fr for BOTH citation and document_date:", df_fr.duplicated(subset=['citation', 'document_date']).sum())

# Blunt removal of duplicates in df_en and df_fr
df_en = df_en.drop_duplicates(subset=['citation']).reset_index(drop=True)
df_fr = df_fr.drop_duplicates(subset=['citation']).reset_index(drop=True)
# Print the number of rows after removing duplicates
print("Number of rows in df_en after removing all duplicate citations:", len(df_en))
print("Number of rows in df_fr after removing duplicate citations:", len(df_fr))

# number of rows in df_en that does not have an identical citation in df_fr
print(
    "Num rows in df_en with no citation match in df_fr",
    df_en[~df_en['citation'].isin(df_fr['citation'])].reset_index(drop=True).shape[0]
)
# number of rows in df_fr that does not have an identical citation in df_en
print(
    "Num rows in df_fr with no citation match in df_en",
    df_fr[~df_fr['citation'].isin(df_en['citation'])].reset_index(drop=True).shape[0]
)



# for df_en add "_en" to end of all cols
df_en.columns = [col + "_en" for col in df_en.columns]
# for df_fr add "_fr" to end of all cols
df_fr.columns = [col + "_fr" for col in df_fr.columns]

# merge df_en and df_fr on citation_en and citation_fr, keeping only matches
df = pd.merge(df_en, df_fr, left_on='citation_en', right_on='citation_fr', how='inner')

# rename dataset_en to dataset, and drop unused cols
df = df.rename(columns={'dataset_en': 'dataset'})
df = df.drop(columns=['dataset_fr', 'language_en', 'language_fr'])


# print len of final df
print("Number of rows in final df:", len(df))
print("*********** AT SOME POINT RECONSIDER THIS ***********")
df.head(5)

Number of duplicated citations in df_en: 111
Number of duplicated citations in df_fr: 108
Number of duplicated rows in df_en for BOTH citation and document_date: 2
Number of duplicated rows in df_fr for BOTH citation and document_date: 1
Number of rows in df_en after removing all duplicate citations: 6579
Number of rows in df_fr after removing duplicate citations: 6564
Num rows in df_en with no citation match in df_fr 60
Num rows in df_fr with no citation match in df_en 45
Number of rows in final df: 6519
*********** AT SOME POINT RECONSIDER THIS ***********


,citation_en,citation2_en,dataset,name_en,language_en,document_date_en,url_en,scraped_timestamp_en,referrer_en,unofficial_text_en,citation_fr,citation2_fr,name_fr,language_fr,document_date_fr,url_fr,scraped_timestamp_fr,referrer_fr,unofficial_text_fr
0,TA6-12745,,RPD,,en,2008-07-03 00:00:00+00:00,1005377.txt,2023-11-12 22:50:45.398529+00:00,a2aj_parse_rpd_txt.ipynb,\nRPD File No. / N° de dossier de la SPR : TA6...,TA6-12745,,,fr,2008-07-03 00:00:00+00:00,1006316.txt,2023-11-12 22:50:56.899575+00:00,a2aj_parse_rpd_txt.ipynb,\nNo de dossier de la SPR/RPD File No.: TA6-12...
1,TA7-02522,,RPD,,en,2009-09-17 00:00:00+00:00,1005448.txt,2023-11-12 22:50:45.662073+00:00,a2aj_parse_rpd_txt.ipynb,\nRPD File No. / N° de dossier de la SPR : TA7...,TA7-02522,,,fr,2009-09-17 00:00:00+00:00,1005526.txt,2023-11-12 22:50:46.012152+00:00,a2aj_parse_rpd_txt.ipynb,\nN° de dossier de la SPR/RPD File No.: TA7-02...
2,MA8-00814,,RPD,,en,2009-09-23 00:00:00+00:00,1005529.txt,2023-11-12 22:50:46.139186+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and\nRefugee Board\nRefugee Protec...,MA8-00814,,,fr,2009-09-23 00:00:00+00:00,1005984.txt,2023-11-12 22:50:54.774444+00:00,a2aj_parse_rpd_txt.ipynb,Commission de l'immigration\net du statut de r...
3,TA8-21155,,RPD,,en,2009-10-27 00:00:00+00:00,1005590.txt,2023-11-12 22:50:46.596770+00:00,a2aj_parse_rpd_txt.ipynb,\nRPD File No. / N° de dossier de la SPR : TA8...,TA8-21155,,,fr,2009-10-27 00:00:00+00:00,1005397.txt,2023-11-12 22:50:45.497039+00:00,a2aj_parse_rpd_txt.ipynb,\nN° de dossier de la SPR/RPD File No.: TA8-21...
4,MA6-00156,,RPD,,en,2008-06-06 00:00:00+00:00,1005614.txt,2023-11-12 22:50:46.774803+00:00,a2aj_parse_rpd_txt.ipynb,Immigration and\nRefugee Board\nRefugee Protec...,MA6-00156,,,fr,2008-06-06 00:00:00+00:00,1005435.txt,2023-11-12 22:50:45.576555+00:00,a2aj_parse_rpd_txt.ipynb,Commission de l'immigration\net du statut de r...


In [14]:
# how many rows in df have document_date_en and document_date_fr that are not the same
print("Number of rows in df with different document_date_en and document_date_fr:", 
      df[df['document_date_en'] != df['document_date_fr']].shape[0])

Number of rows in df with different document_date_en and document_date_fr: 0


In [ ]:
df[df['document_date_en'] != df['document_date_fr']]

In [27]:
# are there any duplicate citations_en in df?
print("Number of duplicated citations_en in df:", df.citation_en.duplicated().sum())
# are there any duplicate citations_fr in df?
print("Number of duplicated citations_fr in df:", df.citation_fr.duplicated().sum())

# are there any blank or null or nan citations_en in df?
print("Number of blank or null or nan citations_en in df:", df['citation_en'].isnull().sum())
# are there any blank or null or nan citations_fr in df?
print("Number of blank or null or nan citations_fr in df:", df['citation_fr'].isnull().sum())

Number of duplicated citations_en in df: 0
Number of duplicated citations_fr in df: 0
Number of blank or null or nan citations_en in df: 0
Number of blank or null or nan citations_fr in df: 0


In [ ]:
# # To compare duplicates
# import difflib

# # Get the two texts
# texts = df_fr[df_fr.duplicated(subset=['citation', 'document_date'], keep=False)].unofficial_text.iloc[[0,1]]

# # Compute the diff
# diff = difflib.unified_diff(
#     texts.iloc[0].splitlines(),
#     texts.iloc[1].splitlines(),
#     fromfile='text0',
#     tofile='text1',
#     lineterm=''
# )

# # Print the diff
# for line in diff:
#     print(line)

### Exports

In [ ]:
# export cleaned df to jsonl
out_path_parsed = pathlib.Path('DATA/rpd_cases.jsonl')
df.to_json(out_path_parsed, orient='records', lines=True)

# export cleaned df to parquet
out_path_parsed = pathlib.Path('DATA/rpd_cases.parquet')

In [24]:
# export to Mongo # NEED TAILSCALE ON
from pymongo import MongoClient
from datetime import timezone
import numpy as np

MONGO_URL = 'mongodb://rll-tr7970x:27017/'
MONGO_DB = 'case-scraping'
MONGO_COLLECTION = 'rpd-legacy-scrapes'
client = MongoClient(MONGO_URL, tz_aware=True, tzinfo=timezone.utc)
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION]

# drop existing collection if it exists
if collection.count_documents({}) > 0:
    collection.drop()

def strip_nulls(recs):
    return [{k: v for k, v in rec.items() if v is not None} for rec in recs]

def sanitize_records(recs):
    """Return a NEW list of dicts that PyMongo can store."""
    return [
        {k: _clean_scalar(v) for k, v in rec.items()}
        for rec in recs
    ]

def _clean_scalar(v):
    # 1) All “missing” markers: <NA>, NaN, NaT  →  None
    if pd.isna(v):
        return None
    
    # 2) pandas Timestamp  →  native datetime with UTC tzinfo
    if isinstance(v, pd.Timestamp):
        return v.to_pydatetime().astimezone(timezone.utc)
    
    # 3) NumPy scalar (np.int64, np.float64, …)  →  Python int/float
    if isinstance(v, np.generic):
        return v.item()
    
    return v

records = df.to_dict(orient="records")
clean_docs = strip_nulls(sanitize_records(records))
result = collection.insert_many(clean_docs)
